In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from collections import Counter
from sklearn import feature_extraction, model_selection, naive_bayes, metrics, svm
from IPython.display import Image
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline  
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Dogrusal olmayan vektörler için, kernel metodu kullanılır. En çok kullanılan kernel modelleri:
*Polynomial Kernel
*Gaussian RBF (Radial Basis Function) Kernel


In [ ]:
data = pd.read_csv('../input/sms-spam-collection-dataset/spam.csv', encoding='latin-1')
data.head()

In [ ]:
#v1 sütunu göre listeleme
count_Class=pd.value_counts(data["v1"], sort= True)
count_Class.plot(kind= 'bar', color= ["blue", "red"])
plt.title('Bar chart')
plt.show()

In [ ]:
#pasta grafigi
count_Class.plot(kind = 'pie',  autopct='%1.00f%%')
plt.title('Pie chart')
plt.ylabel('')
plt.show()

In [ ]:
#list to string
def listToString(s):  
    
    # initialize an empty string 
    str1 = ""  
    
    # traverse in the string   
    for ele in s:  
        str1 += ele   
    
    # return string   
    return str1  

In [ ]:
#en cok tekrarlayan kelimeler
from os import path
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re
cp_data=data.v2.to_list()
wordcloud = WordCloud(max_font_size=40, relative_scaling=.5).generate(listToString(cp_data))
plt.figure()
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
count2 = Counter(" ".join(data[data['v1']=='spam']["v2"]).split()).most_common(20)
df2 = pd.DataFrame.from_dict(count2)
df2 = df2.rename(columns={0: "spam", 1 : "count"})
print(df2.head())
df2.plot.bar(legend = False, color = 'blue')
y_pos = np.arange(len(df2["spam"]))
plt.xticks(y_pos, df2["spam"])
plt.title('En cok tekrar eden spam')
plt.xlabel('kelimeler')
plt.ylabel('sayı')
plt.show()

In [ ]:
 #trainnig
df=data
y=df["v1"]
X=df.drop(["v1"],axis=1)
X.head()

In [ ]:
X = pd.get_dummies(X)
X.head()

In [ ]:
from sklearn.svm import SVC
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.30, random_state=42)

model=SVC(C=1)

model.fit(X_train,y_train)

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
from sklearn import metrics
metrics.confusion_matrix(y_true=y_test,y_pred=y_pred)
#TN FP
#FN TP

#Accuracly: True All/(Total)
#Recall: Tp/Tp+Fn
#Precision: Tp+Fp

In [ ]:
print("accuracy:",metrics.accuracy_score(y_test,y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
print(classification_report(y_test,y_pred))

In [ ]:
#HyperParameter Tuning K-Fold Cross Validation
from sklearn.model_selection import KFold
folds = KFold(n_splits=5,shuffle=True,random_state=4)

model=SVC(C=1)

In [ ]:
from sklearn.model_selection import cross_val_score
model_result=cross_val_score(model,X_train,y_train, cv=folds, scoring='accuracy')
model_result

In [ ]:
print(model_result)
print("mean accuracy: {}".format(model_result.mean()))

In [ ]:
from sklearn.model_selection import GridSearchCV
#Optimal search GridSearch
params={"C":[0.5,1,2,25,50]}

model=SVC()
model_cv=GridSearchCV(estimator=model, param_grid=params,scoring='accuracy',cv=folds,verbose=1,return_train_score=True)
model_cv

In [ ]:
model_cv.fit(X_train,y_train)

In [ ]:
cv_results=pd.DataFrame(model_cv.cv_results_)
cv_results

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(cv_results['param_C'],cv_results['mean_test_score'])
plt.plot(cv_results['param_C'],cv_results['mean_train_score'])
plt.xlabel('C')
plt.ylabel('Accuracy')
plt.legend(['test accuracy','train accuracy'],loc='upper right')
plt.xscale(log)

### optimum degere göre yeniden eğitilebilir

In [ ]:
#Non Linear icin kernel kullanılı

In [ ]:
#svc = svm.SVC(kernel='linear', C=C).fit(X, y)
#rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=C).fit(X, y)
#poly_svc = svm.SVC(kernel='poly', degree=3, C=C).fit(X, y)

 probleme, veri setine göre değişiklik gösteren parametreler hiper-parametre (hyperparameters) olarak adlandırılmaktadır.
 
*  Veri Setinin Boyutu - çeşitliği



In [ ]:
from sklearn.datasets.samples_generator import make_blobs
X, y = make_blobs(n_samples=50, centers=2,
                  random_state=0, cluster_std=0.60)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn');

In [ ]:
from sklearn.svm import SVC # "Support vector classifier"
model = SVC(kernel='linear', C=1E10)
model.fit(X, y)

In [ ]:
def plot_svc_decision_function(model, ax=None, plot_support=True):
    """Plot the decision function for a 2D SVC"""
    if ax is None:
        ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # create grid to evaluate model
    x = np.linspace(xlim[0], xlim[1], 30)
    y = np.linspace(ylim[0], ylim[1], 30)
    Y, X = np.meshgrid(y, x)
    xy = np.vstack([X.ravel(), Y.ravel()]).T
    P = model.decision_function(xy).reshape(X.shape)
    
    # plot decision boundary and margins
    ax.contour(X, Y, P, colors='k',
               levels=[-1, 0, 1], alpha=0.5,
               linestyles=['--', '-', '--'])
    
    # plot support vectors
    if plot_support:
        ax.scatter(model.support_vectors_[:, 0],
                   model.support_vectors_[:, 1],
                   s=300, linewidth=1, facecolors='none');
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
plot_svc_decision_function(model);

In [ ]:
kernels = [‘linear’, ‘rbf’, ‘poly’]for kernel in kernels:
  svc = svm.SVC(kernel=kernel).fit(X, y)
  plotSVC(‘kernel=’ + str(kernel))

In [ ]:
from sklearn.svm import SVC # "Support vector classifier"
rbf_svc = svm.SVC(kernel='rbf', gamma=0.4, C=1).fit(X, y)
model.fit(X, y)